# 🏈 Team Lineup Optimizer

Generates optimal starting lineups for each team in the league based on:
- Player projections (mu, sigma, var, n_sources)
- Roster composition and bye weeks
- Benchmark waiver wire replacements

## Roster Configuration
- 1 QB
- 2 RB
- 2 WR
- 1 TE
- 1 FLEX (WR/RB/TE)
- 1 K
- 1 DST

## Benchmark Replacements
Players below these thresholds are replaced with "Waiver Pickup":
- QB: 18th ranked
- RB: 45th ranked
- WR: 55th ranked
- TE: 18th ranked
- K: 12th ranked
- DEF: 12th ranked


## 📦 Setup


In [1]:
import sqlite3
import sys
import pandas as pd
import json
import ast
from typing import List, Dict, Tuple
import os
from pathlib import Path
from dotenv import load_dotenv

# Add parent directory to path for imports
sys.path.append(str(Path().absolute().parent.parent))

# Load environment variables from root
load_dotenv(Path().absolute().parent.parent / '.env')

# Configuration
LEAGUE_ID = os.getenv("LEAGUE_ID", "1226433368405585920")
SEASON = "2025"
CURRENT_WEEK = 10

# Benchmark thresholds (rank at which we consider waiver wire replacement)
BENCHMARKS = {
    'QB': 18,
    'RB': 40,
    'WR': 50,
    'TE': 22,
    'K': 15,
    'DEF': 15  # Will map to D/ST in projections
}

# Roster slots
ROSTER_SLOTS = {
    'QB': 1,
    'RB': 2,
    'WR': 2,
    'TE': 1,
    'FLEX': 1,  # WR/RB/TE
    'K': 1,
    'DEF': 1
}

# Injury statuses that exclude players from lineups
EXCLUDED_INJURY_STATUSES = ['Out', 'IR', 'PUP', 'Suspended', 'Doubtful']

print("✓ Configuration loaded")
print(f"League ID: {LEAGUE_ID}")
print(f"Season: {SEASON}, Week: {CURRENT_WEEK}")

# Get absolute paths to databases
NOTEBOOK_DIR = Path().absolute()
BACKEND_DIR = NOTEBOOK_DIR.parent
DB_PROJ_PATH = str(BACKEND_DIR / "data" / "databases" / "projections.db")
DB_LEAGUE_PATH = str(BACKEND_DIR / "data" / "databases" / "league.db")


✓ Configuration loaded
League ID: 1226433368405585920
Season: 2025, Week: 10


## 📊 Load Data


In [2]:
# Connect to databases
conn_league = sqlite3.connect(DB_LEAGUE_PATH)
conn_proj = sqlite3.connect(DB_PROJ_PATH)

# Load rosters
query_rosters = f"""
    SELECT 
        r.roster_id,
        r.league_id,
        r.owner_id,
        r.team_name,
        r.players,
        r.wins,
        r.losses,
        r.ties,
        r.fpts,
        u.display_name,
        u.username
    FROM rosters r
    LEFT JOIN users u ON r.owner_id = u.user_id
    WHERE r.league_id = '{LEAGUE_ID}'
    ORDER BY r.roster_id
"""
df_rosters = pd.read_sql_query(query_rosters, conn_league)

print(f"✓ Loaded {len(df_rosters)} teams")
print(f"\nTeams:")
for _, row in df_rosters.iterrows():
    team_name = row['team_name'] or f"Team {row['roster_id']}"
    owner = row['display_name'] or row['username'] or 'Unknown'
    record = f"{row['wins']}-{row['losses']}"
    print(f"  {row['roster_id']:2}. {team_name} ({owner}) - {record}")


✓ Loaded 12 teams

Teams:
   1. Team 1 (xavierking4) - 7-2
   2. Team 2 (asadrafique) - 5-4
   3. Team 3 (amir812) - 5-4
   4. Team 4 (umarrahman30) - 1-8
   5. Team 5 (TBK41) - 5-4
   6. Team 6 (Jibraan) - 4-5
   7. Team 7 (mehdidrissi) - 4-5
   8. Team 8 (sahirsyed30) - 7-2
   9. Team 9 (Bilal879) - 2-7
  10. Team 10 (monkeyman966699696) - 5-4
  11. Team 11 (Ammady) - 4-5
  12. Team 12 (sfaizi24) - 5-4


In [3]:
# Load player projections for current week
query_projections = f"""
    SELECT 
        sleeper_player_id,
        player_name,
        position,
        week,
        mu,
        sigma,
        var,
        n_sources
    FROM player_week_stats
    WHERE week = {CURRENT_WEEK}
"""
df_projections = pd.read_sql_query(query_projections, conn_proj)

# Map DST to DEF for consistency (Sleeper stores defenses as DST)
df_projections['position'] = df_projections['position'].replace('DST', 'DEF')

print(f"✓ Loaded projections for {len(df_projections)} players")
print(f"\nTop 5 projected players:")
print(df_projections.nlargest(5, 'mu')[['player_name', 'position', 'mu', 'sigma']].to_string(index=False))


✓ Loaded projections for 537 players

Top 5 projected players:
        player_name position     mu     sigma
         Josh Allen       QB 23.672  7.415368
Christian McCaffrey       RB 23.274  9.209100
      Lamar Jackson       QB 23.096  7.585006
         Puka Nacua       WR 22.096 10.312726
    Jonathan Taylor       RB 21.218  9.237991


In [4]:
# Load bye weeks for current week
query_byes = f"""
    SELECT DISTINCT team
    FROM nfl_schedules
    WHERE season = '{SEASON}' AND week = {CURRENT_WEEK} AND is_bye = 1
"""
df_byes = pd.read_sql_query(query_byes, conn_league)
bye_teams = set(df_byes['team'].tolist())

print(f"✓ Teams on bye in Week {CURRENT_WEEK}: {', '.join(sorted(bye_teams)) if bye_teams else 'None'}")


✓ Teams on bye in Week 10: CIN, DAL, KC, TEN


In [5]:
# Load NFL players to get team affiliations and injury status
query_nfl_players = """
    SELECT 
        player_id,
        full_name,
        team,
        position,
        injury_status
    FROM nfl_players
"""
df_nfl_players = pd.read_sql_query(query_nfl_players, conn_league)

# Create lookup dictionaries
player_team_map = dict(zip(df_nfl_players['player_id'], df_nfl_players['team']))
player_injury_map = dict(zip(df_nfl_players['player_id'], df_nfl_players['injury_status']))

print(f"✓ Loaded {len(df_nfl_players)} NFL players")
print(f"✓ Injury statuses loaded")


✓ Loaded 3968 NFL players
✓ Injury statuses loaded


## 🎯 Calculate Benchmark Values

Determine the projected points for benchmark players at each position.


In [6]:
# Calculate benchmark values (the mu value at the benchmark rank for each position)
benchmark_values = {}

print("Calculating benchmark replacement values...\n")
print(f"{'Position':<8} {'Rank':<6} {'Player':<25} {'Mu':>8} {'Sigma':>8}")
print("-" * 60)

for position, rank in BENCHMARKS.items():
    pos_players = df_projections[df_projections['position'] == position].copy()
    pos_players = pos_players.sort_values('mu', ascending=False).reset_index(drop=True)
    
    if len(pos_players) >= rank:
        benchmark_player = pos_players.iloc[rank - 1]  # rank is 1-indexed
        benchmark_values[position] = {
            'mu': benchmark_player['mu'],
            'sigma': benchmark_player['sigma'],
            'var': benchmark_player['var'],
            'n_sources': benchmark_player['n_sources'],
            'player_name': benchmark_player['player_name']
        }
        print(f"{position:<8} {rank:<6} {benchmark_player['player_name']:<25} {benchmark_player['mu']:>8.2f} {benchmark_player['sigma']:>8.2f}")
    else:
        # Not enough players, use the lowest available
        if len(pos_players) > 0:
            benchmark_player = pos_players.iloc[-1]
            benchmark_values[position] = {
                'mu': benchmark_player['mu'],
                'sigma': benchmark_player['sigma'],
                'var': benchmark_player['var'],
                'n_sources': benchmark_player['n_sources'],
                'player_name': benchmark_player['player_name']
            }
            print(f"{position:<8} {len(pos_players):<6} {benchmark_player['player_name']:<25} {benchmark_player['mu']:>8.2f} {benchmark_player['sigma']:>8.2f} (Only {len(pos_players)} available)")
        else:
            benchmark_values[position] = {
                'mu': 0.0,
                'sigma': 0.0,
                'var': 0.0,
                'n_sources': 0,
                'player_name': 'No players available'
            }
            print(f"{position:<8} {rank:<6} {'No players available':<25} {0.0:>8.2f} {0.0:>8.2f}")

print("\n✓ Benchmark values calculated")


Calculating benchmark replacement values...

Position Rank   Player                          Mu    Sigma
------------------------------------------------------------
QB       18     Brock Purdy                  15.91    13.00
RB       40     Zonovan Knight                7.36    10.32
WR       50     Rashod Bateman                8.15    10.25
TE       22     Evan Engram                   8.14     8.06
K        15     Jake Elliott                  7.83     4.54
DEF      15     CHI Defense                   5.80     8.49

✓ Benchmark values calculated


## 🔧 Helper Functions


In [7]:
def parse_roster_players(players_str):
    """Parse the players column from roster data."""
    if pd.isna(players_str) or players_str == '':
        return []
    try:
        if isinstance(players_str, str):
            return ast.literal_eval(players_str)
        return players_str
    except:
        return []

def is_on_bye(player_id, player_team_map, bye_teams):
    """Check if a player is on bye this week."""
    team = player_team_map.get(player_id, None)
    return team in bye_teams if team else False

def is_injured_out(player_id, player_injury_map):
    """Check if a player has an injury status that excludes them from lineup."""
    injury_status = player_injury_map.get(player_id, None)
    if injury_status is None:
        return False
    return injury_status in EXCLUDED_INJURY_STATUSES

def get_player_projection(player_id, df_projections):
    """Get projection for a specific player."""
    player_proj = df_projections[df_projections['sleeper_player_id'] == player_id]
    if len(player_proj) > 0:
        return player_proj.iloc[0]
    return None

print("✓ Helper functions defined")


✓ Helper functions defined


In [8]:
def optimize_lineup(roster_players, df_projections, player_team_map, bye_teams, player_injury_map, benchmark_values):
    """
    Optimize lineup for a team's roster.
    Returns a list of lineup slots with player info.
    """
    # Get projections for all roster players
    roster_data = []
    for player_id in roster_players:
        proj = get_player_projection(player_id, df_projections)
        if proj is not None:
            # Skip players who are injured out
            if is_injured_out(player_id, player_injury_map):
                continue
            
            on_bye = is_on_bye(player_id, player_team_map, bye_teams)
            roster_data.append({
                'player_id': player_id,
                'player_name': proj['player_name'],
                'position': proj['position'],
                'mu': proj['mu'] if not on_bye else 0.0,  # 0 points if on bye
                'sigma': proj['sigma'],
                'var': proj['var'],
                'n_sources': proj['n_sources'],
                'on_bye': on_bye
            })
    
    # Sort by mu (descending)
    roster_data.sort(key=lambda x: x['mu'], reverse=True)
    
    # Initialize lineup
    lineup = []
    used_players = set()
    
    # Fill required positions
    for position in ['QB', 'K', 'DEF']:
        count = ROSTER_SLOTS.get(position, 0)
        pos_players = [p for p in roster_data if p['position'] == position and p['player_id'] not in used_players]
        
        for i in range(count):
            if i < len(pos_players):
                player = pos_players[i]
                # Check if below benchmark
                if player['mu'] < benchmark_values[position]['mu'] and not player['on_bye']:
                    # Replace with waiver pickup
                    lineup.append({
                        'slot': position,
                        'player_name': 'Waiver Pickup',
                        'position': position,
                        'mu': benchmark_values[position]['mu'],
                        'sigma': benchmark_values[position]['sigma'],
                        'var': benchmark_values[position]['var'],
                        'n_sources': benchmark_values[position]['n_sources'],
                        'is_replacement': True
                    })
                else:
                    lineup.append({
                        'slot': position,
                        'player_name': player['player_name'],
                        'position': player['position'],
                        'mu': player['mu'],
                        'sigma': player['sigma'],
                        'var': player['var'],
                        'n_sources': player['n_sources'],
                        'is_replacement': False
                    })
                    used_players.add(player['player_id'])
            else:
                # No player available, use waiver pickup
                lineup.append({
                    'slot': position,
                    'player_name': 'Waiver Pickup',
                    'position': position,
                    'mu': benchmark_values[position]['mu'],
                    'sigma': benchmark_values[position]['sigma'],
                    'var': benchmark_values[position]['var'],
                    'n_sources': benchmark_values[position]['n_sources'],
                    'is_replacement': True
                })
    
     # Fill RB, WR, TE positions
    for position in ['RB', 'WR', 'TE']:
        count = ROSTER_SLOTS.get(position, 0)
        pos_players = [p for p in roster_data if p['position'] == position and p['player_id'] not in used_players]
        
        for i in range(count):
            # Create numbered slot (e.g., RB1, RB2, WR1, WR2, TE1)
            slot_name = f'{position}{i+1}' if count > 1 else position
            if i < len(pos_players):
                player = pos_players[i]
                # Check if below benchmark
                if player['mu'] < benchmark_values[position]['mu'] and not player['on_bye']:
                    # Replace with waiver pickup
                    lineup.append({
                        'slot': position,
                        'player_name': 'Waiver Pickup',
                        'position': position,
                        'mu': benchmark_values[position]['mu'],
                        'sigma': benchmark_values[position]['sigma'],
                        'var': benchmark_values[position]['var'],
                        'n_sources': benchmark_values[position]['n_sources'],
                        'is_replacement': True
                    })
                else:
                    lineup.append({
                        'slot': position,
                        'player_name': player['player_name'],
                        'position': player['position'],
                        'mu': player['mu'],
                        'sigma': player['sigma'],
                        'var': player['var'],
                        'n_sources': player['n_sources'],
                        'is_replacement': False
                    })
                    used_players.add(player['player_id'])
            else:
                # No player available, use waiver pickup
                lineup.append({
                    'slot': position,
                    'player_name': 'Waiver Pickup',
                    'position': position,
                    'mu': benchmark_values[position]['mu'],
                    'sigma': benchmark_values[position]['sigma'],
                    'var': benchmark_values[position]['var'],
                    'n_sources': benchmark_values[position]['n_sources'],
                    'is_replacement': True
                })
    
    # Fill FLEX (best remaining RB/WR/TE)
    flex_candidates = [p for p in roster_data 
                      if p['position'] in ['RB', 'WR', 'TE'] 
                      and p['player_id'] not in used_players]
    
    if len(flex_candidates) > 0:
        player = flex_candidates[0]  # Already sorted by mu
        # Check if below benchmark for their position
        if player['mu'] < benchmark_values[player['position']]['mu'] and not player['on_bye']:
            # Use best benchmark among eligible positions
            best_benchmark_pos = max(['RB', 'WR', 'TE'], 
                                    key=lambda p: benchmark_values[p]['mu'])
            lineup.append({
                'slot': 'FLEX',
                'player_name': 'Waiver Pickup',
                'position': best_benchmark_pos,
                'mu': benchmark_values[best_benchmark_pos]['mu'],
                'sigma': benchmark_values[best_benchmark_pos]['sigma'],
                'var': benchmark_values[best_benchmark_pos]['var'],
                'n_sources': benchmark_values[best_benchmark_pos]['n_sources'],
                'is_replacement': True
            })
        else:
            lineup.append({
                'slot': 'FLEX',
                'player_name': player['player_name'],
                'position': player['position'],
                'mu': player['mu'],
                'sigma': player['sigma'],
                'var': player['var'],
                'n_sources': player['n_sources'],
                'is_replacement': False
            })
            used_players.add(player['player_id'])
    else:
        # No flex candidate, use best benchmark
        best_benchmark_pos = max(['RB', 'WR', 'TE'], 
                                key=lambda p: benchmark_values[p]['mu'])
        lineup.append({
            'slot': 'FLEX',
            'player_name': 'Waiver Pickup',
            'position': best_benchmark_pos,
            'mu': benchmark_values[best_benchmark_pos]['mu'],
            'sigma': benchmark_values[best_benchmark_pos]['sigma'],
            'var': benchmark_values[best_benchmark_pos]['var'],
            'n_sources': benchmark_values[best_benchmark_pos]['n_sources'],
            'is_replacement': True
        })
    
    return lineup

print("✓ Lineup optimization function defined")


✓ Lineup optimization function defined


## 🎮 Generate Optimal Lineups for All Teams


In [9]:
# Generate optimal lineups for all teams
all_lineups = []

print("Generating optimal lineups...\n")

for _, roster_row in df_rosters.iterrows():
    roster_id = roster_row['roster_id']
    team_name = roster_row['team_name'] or f"Team {roster_id}"
    owner = roster_row['display_name'] or roster_row['username'] or 'Unknown'
    
    # Parse roster players
    roster_players = parse_roster_players(roster_row['players'])
    
    print(f"Team {roster_id}: {team_name} ({owner})")
    print(f"  Roster size: {len(roster_players)} players")
    
    # Optimize lineup
    lineup = optimize_lineup(roster_players, df_projections, player_team_map, bye_teams, player_injury_map, benchmark_values)
    
    # Add to results
    for slot_data in lineup:
        all_lineups.append({
            'roster_id': roster_id,
            'team_name': team_name,
            'owner': owner,
            'record': f"{roster_row['wins']}-{roster_row['losses']}",
            'slot': slot_data['slot'],
            'player_name': slot_data['player_name'],
            'position': slot_data['position'],
            'mu': slot_data['mu'],
            'sigma': slot_data['sigma'],
            'var': slot_data['var'],
            'n_sources': slot_data['n_sources'],
            'is_replacement': slot_data['is_replacement']
        })
    
    total_mu = sum(s['mu'] for s in lineup)
    print(f"  Projected total: {total_mu:.2f} points")
    print()

# Create DataFrame
df_lineups = pd.DataFrame(all_lineups)

print(f"✓ Generated lineups for {len(df_rosters)} teams")
print(f"✓ Total lineup slots: {len(df_lineups)}")


Generating optimal lineups...

Team 1: Team 1 (xavierking4)
  Roster size: 15 players
  Projected total: 113.62 points

Team 2: Team 2 (asadrafique)
  Roster size: 15 players
  Projected total: 109.50 points

Team 3: Team 3 (amir812)
  Roster size: 14 players
  Projected total: 125.28 points

Team 4: Team 4 (umarrahman30)
  Roster size: 15 players
  Projected total: 110.68 points

Team 5: Team 5 (TBK41)
  Roster size: 12 players
  Projected total: 113.84 points

Team 6: Team 6 (Jibraan)
  Roster size: 14 players
  Projected total: 125.67 points

Team 7: Team 7 (mehdidrissi)
  Roster size: 13 players
  Projected total: 119.24 points

Team 8: Team 8 (sahirsyed30)
  Roster size: 14 players
  Projected total: 107.73 points

Team 9: Team 9 (Bilal879)
  Roster size: 14 players
  Projected total: 110.63 points

Team 10: Team 10 (monkeyman966699696)
  Roster size: 15 players
  Projected total: 118.36 points

Team 11: Team 11 (Ammady)
  Roster size: 14 players
  Projected total: 116.63 points



## 📊 Complete Lineup Table

All simulated starting lineups with projections


In [10]:
# Display complete table
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Format for display
df_display = df_lineups.copy()
df_display['mu'] = df_display['mu'].round(2)
df_display['sigma'] = df_display['sigma'].round(2)
df_display['var'] = df_display['var'].round(2)

# Mark replacements with text suffix
df_display['player_display'] = df_display.apply(
    lambda x: f"{x['player_name']} [WAIVER]" if x['is_replacement'] else x['player_name'], 
    axis=1
)

# Select columns for display
df_display_final = df_display[[
    'roster_id', 'team_name', 'owner', 'record', 'slot', 
    'player_display', 'position', 'mu', 'sigma', 'var', 'n_sources'
]].copy()

df_display_final.columns = [
    'Team ID', 'Team Name', 'Owner', 'Record', 'Slot',
    'Player', 'Pos', 'Mu', 'Sigma', 'Var', 'Sources'
]

print("="*120)
print("SIMULATED STARTING LINEUPS - All Teams")
print(f"Week {CURRENT_WEEK} Projections")
print("[WAIVER] = Waiver Pickup (Benchmark Replacement)")
print("="*120)
print()

print(df_display_final.to_string(index=False))

print()
print("="*120)


SIMULATED STARTING LINEUPS - All Teams
Week 10 Projections
[WAIVER] = Waiver Pickup (Benchmark Replacement)

 Team ID Team Name              Owner Record Slot                 Player Pos    Mu  Sigma    Var  Sources
       1    Team 1        xavierking4    7-2   QB Waiver Pickup [WAIVER]  QB 15.90  13.00 168.93        4
       1    Team 1        xavierking4    7-2    K           Cairo Santos   K  8.47   4.03  16.21        3
       1    Team 1        xavierking4    7-2  DEF             NE Defense DEF  5.90   7.91  62.52        2
       1    Team 1        xavierking4    7-2   RB         Bijan Robinson  RB 20.69   9.15  83.77        5
       1    Team 1        xavierking4    7-2   RB Jacory Croskey-Merritt  RB  9.74   9.23  85.17        5
       1    Team 1        xavierking4    7-2   WR           Emeka Egbuka  WR 15.65  10.07 101.33        5
       1    Team 1        xavierking4    7-2   WR       Jameson Williams  WR 11.30  10.12 102.36        5
       1    Team 1        xavierking4    7-

## 📈 Team Projections Summary


In [11]:
# Calculate team totals
team_totals = df_lineups.groupby(['roster_id', 'team_name', 'owner', 'record']).agg({
    'mu': 'sum',
    'sigma': lambda x: (sum(x**2))**0.5,  # Combined sigma (assuming independence)
    'var': 'sum',  # Variance is additive for independent variables
    'is_replacement': 'sum'  # Count replacements
}).reset_index()

team_totals.columns = ['Team ID', 'Team Name', 'Owner', 'Record', 'Total Mu', 'Combined Sigma', 'Total Var', 'Waiver Pickups']
team_totals = team_totals.sort_values('Total Mu', ascending=False)

team_totals['Total Mu'] = team_totals['Total Mu'].round(2)
team_totals['Combined Sigma'] = team_totals['Combined Sigma'].round(2)
team_totals['Total Var'] = team_totals['Total Var'].round(2)
team_totals['Waiver Pickups'] = team_totals['Waiver Pickups'].astype(int)

print("="*100)
print("TEAM PROJECTIONS SUMMARY")
print(f"Week {CURRENT_WEEK} - Ranked by Projected Points")
print("="*100)
print()

print(team_totals.to_string(index=False))

print()
print("="*100)
print(f"\nAverage projected points: {team_totals['Total Mu'].mean():.2f}")
print(f"Median projected points: {team_totals['Total Mu'].median():.2f}")
print(f"Highest projection: {team_totals['Total Mu'].max():.2f}")
print(f"Lowest projection: {team_totals['Total Mu'].min():.2f}")


TEAM PROJECTIONS SUMMARY
Week 10 - Ranked by Projected Points

 Team ID Team Name              Owner Record  Total Mu  Combined Sigma  Total Var  Waiver Pickups
       6    Team 6            Jibraan    4-5    125.67           26.29     691.21               0
       3    Team 3            amir812    5-4    125.28           28.04     786.08               3
       7    Team 7        mehdidrissi    4-5    119.24           26.40     697.02               1
      10   Team 10 monkeyman966699696    5-4    118.36           26.34     693.65               0
      11   Team 11             Ammady    4-5    116.63           26.32     692.85               1
       5    Team 5              TBK41    5-4    113.84           26.51     702.81               2
       1    Team 1        xavierking4    7-2    113.62           28.13     791.24               1
      12   Team 12           sfaizi24    5-4    110.95           26.78     717.32               2
       4    Team 4       umarrahman30    1-8    110.68 

## 💾 Export Results


In [12]:
# Export to CSV
output_file = f'../data/csv/team_lineups_week_{CURRENT_WEEK}.csv'
df_lineups.to_csv(output_file, index=False)

summary_file = f'../data/csv/team_projections_summary_week_{CURRENT_WEEK}.csv'
team_totals.to_csv(summary_file, index=False)

print(f"✓ Exported detailed lineups to: {output_file}")
print(f"✓ Exported summary to: {summary_file}")

# Export to database
print("\nExporting to database...")

# Prepare lineup data for database (add week and timestamp)
df_lineups_db = df_lineups.copy()
df_lineups_db['week'] = CURRENT_WEEK
df_lineups_db['season'] = SEASON
df_lineups_db['timestamp'] = pd.Timestamp.now().isoformat()

# Prepare team summary data for database
team_totals_db = team_totals.copy()
team_totals_db['week'] = CURRENT_WEEK
team_totals_db['season'] = SEASON
team_totals_db['timestamp'] = pd.Timestamp.now().isoformat()

# Export to projections.db
try:
    conn_proj_write = sqlite3.connect(DB_PROJ_PATH)
    cursor = conn_proj_write.cursor()
    
    # Create team_lineups table if it doesn't exist
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS team_lineups (
            roster_id INTEGER,
            team_name TEXT,
            owner TEXT,
            record TEXT,
            slot TEXT,
            player_name TEXT,
            position TEXT,
            mu REAL,
            sigma REAL,
            var REAL,
            n_sources INTEGER,
            is_replacement INTEGER,
            week INTEGER,
            season TEXT,
            timestamp TEXT,
            PRIMARY KEY (team_name, week, slot)
        )
    """)
    
    # Create team_projections_summary table if it doesn't exist
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS team_projections_summary (
            roster_id INTEGER,
            team_name TEXT,
            owner TEXT,
            record TEXT,
            total_mu REAL,
            combined_sigma REAL,
            total_var REAL,
            waiver_pickups INTEGER,
            week INTEGER,
            season TEXT,
            timestamp TEXT,
            PRIMARY KEY (team_name, week)
        )
    """)
    
    print("✓ Tables created/verified")
    
    # Rename columns in team_totals_db to match database schema
    team_totals_db_renamed = team_totals_db.rename(columns={
        'Team ID': 'roster_id',
        'Team Name': 'team_name',
        'Owner': 'owner',
        'Record': 'record',
        'Total Mu': 'total_mu',
        'Combined Sigma': 'combined_sigma',
        'Total Var': 'total_var',
        'Waiver Pickups': 'waiver_pickups'
    })
    
    # Insert detailed lineups using INSERT OR REPLACE to handle duplicates
    # This will replace old records if they conflict with the primary key
    lineup_columns = ['roster_id', 'team_name', 'owner', 'record', 'slot', 'player_name', 
                      'position', 'mu', 'sigma', 'var', 'n_sources', 'is_replacement', 
                      'week', 'season', 'timestamp']
    
    for _, row in df_lineups_db.iterrows():
        cursor.execute(f"""
            INSERT OR REPLACE INTO team_lineups 
            ({', '.join(lineup_columns)}) 
            VALUES ({', '.join(['?'] * len(lineup_columns))})
        """, tuple(row[col] for col in lineup_columns))
    
    print(f"✓ Exported {len(df_lineups_db)} lineup slots to projections.db (team_lineups table)")
    
    # Insert team summary using INSERT OR REPLACE
    summary_columns = ['roster_id', 'team_name', 'owner', 'record', 'total_mu', 
                       'combined_sigma', 'total_var', 'waiver_pickups', 'week', 'season', 'timestamp']
    
    for _, row in team_totals_db_renamed.iterrows():
        cursor.execute(f"""
            INSERT OR REPLACE INTO team_projections_summary 
            ({', '.join(summary_columns)}) 
            VALUES ({', '.join(['?'] * len(summary_columns))})
        """, tuple(row[col] for col in summary_columns))
    
    print(f"✓ Exported {len(team_totals_db_renamed)} team summaries to projections.db (team_projections_summary table)")
    
    conn_proj_write.commit()
    conn_proj_write.close()
    
    print("✓ Database export complete")
    
except Exception as e:
    print(f"⚠ Database export error: {e}")
    print("  (CSV files were still created successfully)")
    import traceback
    traceback.print_exc()


✓ Exported detailed lineups to: ../data/csv/team_lineups_week_10.csv
✓ Exported summary to: ../data/csv/team_projections_summary_week_10.csv

Exporting to database...
✓ Tables created/verified
✓ Exported 108 lineup slots to projections.db (team_lineups table)
✓ Exported 12 team summaries to projections.db (team_projections_summary table)
✓ Database export complete


## 🔍 Individual Team Deep Dive

View detailed lineup for a specific team


In [13]:
def show_team_lineup(roster_id):
    """Display detailed lineup for a specific team."""
    team_data = df_lineups[df_lineups['roster_id'] == roster_id].copy()
    
    if len(team_data) == 0:
        print(f"Team {roster_id} not found")
        return
    
    team_name = team_data.iloc[0]['team_name']
    owner = team_data.iloc[0]['owner']
    record = team_data.iloc[0]['record']
    
    print("="*90)
    print(f"TEAM {roster_id}: {team_name}")
    print(f"Owner: {owner} | Record: {record}")
    print("="*90)
    print()
    
    print(f"{'Slot':<6} {'Player':<35} {'Pos':<5} {'Mu':>8} {'Sigma':>8} {'Var':>8} {'Src':>4}")
    print("-"*90)
    
    total_mu = 0
    total_var = 0
    
    for _, row in team_data.iterrows():
        marker = " [WAIVER]" if row['is_replacement'] else ""
        player_display = f"{row['player_name']}{marker}"
        print(f"{row['slot']:<6} {player_display:<35} {row['position']:<5} {row['mu']:>8.2f} {row['sigma']:>8.2f} {row['var']:>8.2f} {row['n_sources']:>4}")
        total_mu += row['mu']
        total_var += row['var']
    
    total_sigma = total_var ** 0.5
    
    print("-"*90)
    print(f"{'TOTAL':<6} {'':<35} {'':<5} {total_mu:>8.2f} {total_sigma:>8.2f} {total_var:>8.2f}")
    print()
    print("[WAIVER] = Waiver Pickup (Benchmark Replacement)")
    print("="*90)

# Example: Show lineup for team 1
show_team_lineup(8)


TEAM 8: Team 8
Owner: sahirsyed30 | Record: 7-2

Slot   Player                              Pos         Mu    Sigma      Var  Src
------------------------------------------------------------------------------------------
QB     Jalen Hurts                         QB       20.23     8.01    64.12    5
K      Waiver Pickup [WAIVER]              K         7.83     4.54    20.57    3
DEF    Waiver Pickup [WAIVER]              DEF       5.80     8.49    72.12    2
RB     James Cook                          RB       18.58     9.75    95.09    5
RB     Zonovan Knight                      RB        7.36    10.32   106.44    3
WR     DeVonta Smith                       WR       13.96    10.14   102.89    5
WR     Chris Olave                         WR       13.96    10.16   103.23    5
TE     Waiver Pickup [WAIVER]              TE        8.14     8.06    65.04    5
FLEX   Jordan Addison                      WR       11.87    10.16   103.15    5
--------------------------------------------------

## 🏆 Position Group Analysis


In [14]:
# Analyze by position group
position_analysis = df_lineups.groupby('slot').agg({
    'mu': ['mean', 'min', 'max', 'std'],
    'is_replacement': 'sum'
}).round(2)

position_analysis.columns = ['Avg Mu', 'Min Mu', 'Max Mu', 'Std Dev', 'Replacements']
position_analysis['Replacements'] = position_analysis['Replacements'].astype(int)

print("="*80)
print("POSITION GROUP ANALYSIS")
print(f"Across all {len(df_rosters)} teams")
print("="*80)
print()

print(position_analysis)

print()
print("="*80)


POSITION GROUP ANALYSIS
Across all 12 teams

      Avg Mu  Min Mu  Max Mu  Std Dev  Replacements
slot                                               
DEF     6.78    5.80    9.10     1.22             4
FLEX   11.17    8.15   14.11     1.91             1
K       8.40    7.83    9.47     0.60             5
QB     19.29   15.90   23.67     2.56             3
RB     14.44    7.36   23.27     4.47             1
TE     11.20    8.14   15.63     2.60             3
WR     14.73   10.89   22.10     2.99             0



## 🧹 Cleanup


In [15]:
# Close database connections
conn_league.close()
conn_proj.close()

print("✓ Database connections closed")


✓ Database connections closed
